In [ ]:
import json
import os

print("Configurando o ambiente para leitura de arquivos locais...")

PATH_TABLES = 'tables.json'
PATH_TRAIN_DATA = 'train_spider.json'
PATH_DEV_DATA = 'dev.json'

TRAIN_OUTPUT_FILE = 'train_formatted.jsonl'
DEV_OUTPUT_FILE = 'dev_formatted.jsonl'

print("Caminhos definidos.")

Configurando o ambiente para leitura de arquivos locais...
Caminhos definidos.


In [ ]:
def generate_schemas_from_tables_json(path_to_tables_json):
    """Lê o arquivo tables.json e gera um dicionário de schemas."""
    print(f"Gerando dicionário de schemas a partir de '{path_to_tables_json}'...")
    try:
        with open(path_to_tables_json, 'r') as f:
            data = json.load(f)
    except FileNotFoundError:
        print(f"ERRO: Arquivo '{path_to_tables_json}' não encontrado!")
        print("Certifique-se de que você fez o upload do arquivo 'tables.json' para o ambiente.")
        return None

    schemas_dict = {}
    for db_info in data:
        db_id = db_info['db_id']
        table_names = db_info['table_names_original']
        columns_info = db_info['column_names_original']
        column_types = db_info['column_types']
        primary_keys_indices = db_info['primary_keys']
        db_schema_parts = []
        column_details = {i: (columns_info[i][1], column_types[i]) for i in range(len(columns_info))}

        for i, table_name in enumerate(table_names):
            table_creation_string = f"CREATE TABLE `{table_name}` (\n"
            table_columns = []
            for col_idx, (tbl_idx, col_name) in enumerate(columns_info):
                if tbl_idx == i:
                    col_type = column_types[col_idx]
                    table_columns.append(f"  `{col_name}` {col_type.upper()}")
            table_creation_string += ",\n".join(table_columns)
            pk_cols = []
            for pk_index in primary_keys_indices:
                if columns_info[pk_index][0] == i:
                    pk_col_name = column_details[pk_index][0]
                    pk_cols.append(f"`{pk_col_name}`")
            if pk_cols:
                table_creation_string += f",\n  PRIMARY KEY ({', '.join(pk_cols)})"
            table_creation_string += "\n);"
            db_schema_parts.append(table_creation_string)
        schemas_dict[db_id] = "\n".join(db_schema_parts)

    print(f"Dicionário de schemas gerado com sucesso com {len(schemas_dict)} entradas.")
    return schemas_dict

In [ ]:
def format_and_save_local_json(input_json_path, schema_dict, output_file):
    """
    Lê um arquivo JSON local do Spider, formata-o em prompts de texto
    e salva o resultado em um arquivo JSON Lines (.jsonl) com a codificação correta.
    """
    print(f"\n--- Processando o arquivo local: '{input_json_path}' ---")

    try:
        with open(input_json_path, 'r') as f:
            local_data = json.load(f)
    except FileNotFoundError:
        print(f"ERRO: Arquivo de dados '{input_json_path}' não encontrado!")
        print(f"Certifique-se de que você fez o upload do arquivo.")
        return

    PROMPT_TEMPLATE = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>

   You are an expert model specializing in the task of converting natural language text into SQL queries. Your role is to answer user questions by generating syntactically correct SQL queries based on the provided database schema. Your response must be solely the SQL query corresponding to the natural language text. Generate the SQL query in a single line. Do not insert line breaks within the generated query. Use ONLY the tables and columns provided in the ‘Database Schema’ section, referring to them correctly. Do not make up column or table names. Use table aliases (e.g., T1, T2) only when a JOIN operation between multiple tables is required, either to disambiguate columns with the same name or to improve clarity. For queries involving a single table, do not use aliases. Refer to the examples below to understand how you should respond. <|eot_id|><|start_header_id|>user<|end_header_id|>

    ### Examples of expected output:

    1.
      - schema example: 'CREATE TABLE trip (end_station_id VARCHAR); CREATE TABLE station (id VARCHAR, city VARCHAR)'
      - natural language text: 'Count the number of trips that did not end in San Francisco city.'
      - expected SQL query: 'SELECT COUNT(*) FROM trips WHERE end_city != 'San Francisco'
    2.
      - schema example: 'CREATE TABLE Aircraft (aid VARCHAR, distance INTEGER)'
      - natural language text: 'Show ids for all aircrafts with more than 1000 distance.'
      - expected SQL query: 'SELECT aid FROM Aircraft WHERE distance > 1000'
    3.
      - schema example: 'CREATE TABLE actor (Musical_ID VARCHAR); CREATE TABLE musical (Name VARCHAR, Musical_ID VARCHAR)'
      - natural language text: 'Show names of musicals which have at least three actors.'
      - expected SQL query: 'SELECT T2.Name FROM actor AS T1 JOIN musical AS T2 ON T1.Musical_ID = T2.Musical_ID GROUP BY T1.Musical_ID HAVING COUNT(*) >= 3'

    ### Database Schema:
    {schema}

    ### Question:
    {question}

    <|eot_id|><|start_header_id|>assistant<|end_header_id|>

    {sql_query}<|eot_id|>"""


    with open(output_file, 'w', encoding='utf-8') as f_out:
        for example in local_data:
            db_id = example['db_id']
            schema = schema_dict.get(db_id, "")
            if not schema:
                print(f"AVISO: Schema para o db_id '{db_id}' não encontrado.")

            prompt_full = PROMPT_TEMPLATE.format(
                schema=schema,
                question=example['question'],
                sql_query=example['query']
            )

            line_obj = {'text': prompt_full}

            f_out.write(json.dumps(line_obj, ensure_ascii=False) + '\n')

    print(f"Total de exemplos processados: {len(local_data)}")

In [ ]:
master_schema_dict = generate_schemas_from_tables_json(PATH_TABLES)

if master_schema_dict:

    format_and_save_local_json(
        input_json_path=PATH_TRAIN_DATA,
        schema_dict=master_schema_dict,
        output_file=TRAIN_OUTPUT_FILE
    )

    format_and_save_local_json(
        input_json_path=PATH_DEV_DATA,
        schema_dict=master_schema_dict,
        output_file=DEV_OUTPUT_FILE
    )

Gerando dicionário de schemas a partir de 'tables.json'...
Dicionário de schemas gerado com sucesso com 166 entradas.

--- Processando o arquivo local: 'train_spider.json' ---
✅ Arquivo 'train_spider.json' formatado e salvo com sucesso em 'train_formatted.jsonl'
Total de exemplos processados: 7000

--- Processando o arquivo local: 'dev.json' ---
✅ Arquivo 'dev.json' formatado e salvo com sucesso em 'dev_formatted.jsonl'
Total de exemplos processados: 1034

🎉 Processo concluído! Seus arquivos formatados estão prontos.
